### 富文本编辑器
+ 借助富文本编辑器，管理员能够编辑出来一个包含html的页面，从而页面的显示效果，可以由管理员定义，而不用完全依赖于前期开发人员

In [ ]:
# 应用到项目中
# 在settings.py中为INSTALLED_APPS添加编辑器应用
INSTALLED_APPS = (
    ...
    'tinymce',
)
# 在settings.py中添加编辑配置项
TINYMCE_DEFAULT_CONFIG = {
    'theme': 'advanced',
    'width': 600,
    'height': 400,
}
# 在根urls.py中配置
urlpatterns = [
    ...
    url(r'^tinymce/', include('tinymce.urls')),
]
# 在应用中定义模型的属性
from django.db import models
from tinymce.models import HTMLField

class HeroInfo(models.Model):
    ...
    hcontent = HTMLField()

In [ ]:
# 自定义使用
# 定义视图editor，用于显示编辑器并完成提交
def editor(request):
    return render(request, 'other/editor.html')

# 配置url
urlpatterns = [
    ...
    url(r'^editor/$', views.editor, name='editor'),
]

# 创建模板editor.html
<!DOCTYPE html>
<html>
<head>
    <title></title>
    <script type="text/javascript" src='/static/tiny_mce/tiny_mce.js'></script>
    <script type="text/javascript">
        tinyMCE.init({
            'mode':'textareas',
            'theme':'advanced',
            'width':400,
            'height':100
        });
    </script>
</head>
<body>
<form method="post" action="/content/">
    <input type="text" name="hname">
    <br>
    <textarea name='hcontent'>哈哈，这是啥呀</textarea>
    <br>
    <input type="submit" value="提交">
</form>
</body>
</html>

# 定义视图content，接收请求，并更新heroinfo对象
def content(request):
    hname = request.POST['hname']
    hcontent = request.POST['hcontent']

    heroinfo = HeroInfo.objects.get(pk=1)
    heroinfo.hname = hname
    heroinfo.hcontent = hcontent
    heroinfo.save()

    return render(request, 'other/content.html', {'hero': heroinfo})

# 添加url项
urlpatterns = [
    ...
    url(r'^content/$', views.content, name='content'),
]

# 定义模板content.html
<!DOCTYPE html>
<html>
<head>
    <title></title>
</head>
<body>
姓名：{{hero.hname}}
<hr>
{%autoescape off%}
{{hero.hcontent}}
{%endautoescape%}
</body>
</html>

### 全文检索
+ 全文检索不同于特定字段的模糊查询，使用全文检索的效率更高，并且能够对于中文进行分词处理
+ haystack：django的一个包，可以方便地对model里面的内容进行索引、搜索，设计为支持whoosh,solr,Xapian,Elasticsearc四种全文检索引擎后端，属于一种全文检索的框架
+ whoosh：纯Python编写的全文搜索引擎，虽然性能比不上sphinx、xapian、Elasticsearc等，但是无二进制包，程序不会莫名其妙的崩溃，对于小型的站点，whoosh已经足够使用
+ jieba：一款免费的中文分词包，如果觉得不好用可以使用一些收费产品

In [ ]:
# 1.在虚拟环境中依次安装包
pip install django-haystack
pip install whoosh
pip install jieba

In [ ]:
# 2.修改settings.py文件
#添加应用
INSTALLED_APPS = (
    ...
    'haystack',
)
# 添加搜索引擎
HAYSTACK_CONNECTIONS = {
    'default': {
        'ENGINE': 'haystack.backends.whoosh_cn_backend.WhooshEngine',
        'PATH': os.path.join(BASE_DIR, 'whoosh_index'),
    }
}

# 自动生成索引
HAYSTACK_SIGNAL_PROCESSOR = 'haystack.signals.RealtimeSignalProcessor'

In [ ]:
# 3.在项目的urls.py中添加url

urlpatterns = [
    ...
    url(r'^search/', include('haystack.urls')),
]

In [ ]:
# 4.在应用目录下建立search_indexes.py文件
from haystack import indexes
from models import GoodsInfo


class GoodsInfoIndex(indexes.SearchIndex, indexes.Indexable):
    text = indexes.CharField(document=True, use_template=True)

    def get_model(self):
        return GoodsInfo

    def index_queryset(self, using=None):
        return self.get_model().objects.all()

In [ ]:
# 5.在目录“templates/search/indexes/应用名称/”下创建“模型类名称_text.txt”文件
# goodsinfo_text.txt，这里列出了要对哪些列的内容进行检索
{{ object.gName }}
{{ object.gSubName }}
{{ object.gDes }}

In [ ]:
# 6.在目录“templates/search/”下建立search.html
<!DOCTYPE html>
<html>
<head>
    <title></title>
</head>
<body>
{% if query %}
    <h3>搜索结果如下：</h3>
    {% for result in page.object_list %}
        <a href="/{{ result.object.id }}/">{{ result.object.gName }}</a><br/>
    {% empty %}
        <p>啥也没找到</p>
    {% endfor %}

    {% if page.has_previous or page.has_next %}
        <div>
            {% if page.has_previous %}<a href="?q={{ query }}&amp;page={{ page.previous_page_number }}">{% endif %}&laquo; 上一页{% if page.has_previous %}</a>{% endif %}
        |
            {% if page.has_next %}<a href="?q={{ query }}&amp;page={{ page.next_page_number }}">{% endif %}下一页 &raquo;{% if page.has_next %}</a>{% endif %}
        </div>
    {% endif %}
{% endif %}
</body>
</html>

In [ ]:
# 7.建立ChineseAnalyzer.py文件
# 保存在haystack的安装文件夹下，路径如“/home/python/.virtualenvs/django_py2/lib/python2.7/site-packages/haystack/backends”
import jieba
from whoosh.analysis import Tokenizer, Token


class ChineseTokenizer(Tokenizer):
    def __call__(self, value, positions=False, chars=False,
                 keeporiginal=False, removestops=True,
                 start_pos=0, start_char=0, mode='', **kwargs):
        t = Token(positions, chars, removestops=removestops, mode=mode,
                  **kwargs)
        seglist = jieba.cut(value, cut_all=True)
        for w in seglist:
            t.original = t.text = w
            t.boost = 1.0
            if positions:
                t.pos = start_pos + value.find(w)
            if chars:
                t.startchar = start_char + value.find(w)
                t.endchar = start_char + value.find(w) + len(w)
            yield t


def ChineseAnalyzer():
    return ChineseTokenizer()

In [ ]:
# 8.复制whoosh_backend.py文件，改名为whoosh_cn_backend.py
# 注意：复制出来的文件名，末尾会有一个空格，记得要删除这个空格
from .ChineseAnalyzer import ChineseAnalyzer 
# 查找
analyzer=StemmingAnalyzer()
# 改为
analyzer=ChineseAnalyzer()

In [ ]:
# 9.生成索引
# 初始化索引数据
python manage.py rebuild_index

In [ ]:
# 10.在模板中创建搜索栏
<form method='get' action="/search/" target="_blank">
    <input type="text" name="q">
    <input type="submit" value="查询">
</form>

### celery
+ 示例一：用户发起request，并等待response返回。在本些views中，可能需要执行一段耗时的程序，那么用户就会等待很长时间，造成不好的用户体验
+ 示例二：网站每小时需要同步一次天气预报信息，但是http是请求触发的，难道要一小时请求一次吗？
+ 使用celery后，情况就不一样了
+ 示例二的解决：使用celery定时执行

#### 名词
+ 任务task：就是一个Python函数
+ 队列queue：将需要执行的任务加入到队列中
+ 工人worker：在一个新进程中，负责执行队列中的任务
+ 代理人broker：负责调度，在布置环境中使用redis

In [ ]:
# 使用
# 安装包
# celery==3.1.25
# celery-with-redis==3.0
# django-celery==3.1.17

# 配置settings
INSTALLED_APPS = (
  ...
  'djcelery',
}

...

import djcelery
djcelery.setup_loader()
BROKER_URL = 'redis://127.0.0.1:6379/0'
CELERY_IMPORTS = ('应用名称.task')
#在应用目录下创建task.py文件

import time
from celery import task

@task
def sayhello():
    print('hello ...')
    time.sleep(2)
    print('world ...')
# 迁移，生成celery需要的数据表
python manage.py migrate
# 启动Redis
sudo redis-server /etc/redis/redis.conf
# 启动worker
python manage.py celery worker --loglevel=info
# 调用语法
function.delay(parameters)
# 使用代码
#from task import *

def sayhello(request):
    print('hello ...')
    import time
    time.sleep(10)
    print('world ...')

    # sayhello.delay()

    return HttpResponse("hello world")

### 部署
#### 服务器介绍
+ 服务器：私有服务器、公有服务器
+ 私有服务器：公司自己购买、自己维护，只布署自己的应用，可供公司内部或外网访问
+ 公有服务器：集成好运营环境，销售空间或主机，供其布署自己的应用
+ 私有服务器成本高，需要专业人员维护，适合大公司使用
+ 公有服务器适合初创公司使用，成本低
+ 常用的公有服务器，如阿里云、青云等，可根据需要，按流量收费或按时间收费
+ 此处的服务器是物理上的一台非常高、线路全、运行稳定的机器

#### 服务器环境配置
+ 在本地的虚拟环境中，项目根目录下，执行命令收集所有包
+ pip freeze > plist.txt<br><br>

+ 通过ftp软件将开发好的项目上传到此服务器的某个目录
+ 安装并创建虚拟环境，如果已有则跳过此步
+ sudo apt-get install python-virtualenv
+ sudo easy_install virtualenvwrapper
+ mkvirtualenv [虚拟环境名称]<br><br>

+ 在虚拟环境上工作，安装所有需要的包
+ workon [虚拟环境名称]
+ pip install -r plist.txt<br><br>

+ 更改settings.py文件
+ DEBUG = False
+ ALLOW_HOSTS=['*',]表示可以访问服务器的ip
+ 启动服务器，运行正常，但是静态文件无法加载<br><br>

#### WSGI
+ python manage.py runserver：这是一款适合开发阶段使用的服务器，不适合运行在真实的生产环境中
+ 在生产环境中使用WSGI
+ WSGI：Web服务器网关接口，英文为Python Web Server Gateway Interface，缩写为WSGI，是Python应用程序或框架和Web服务器之间的一种接口，被广泛接受
+ WSGI没有官方的实现, 因为WSGI更像一个协议，只要遵照这些协议,WSGI应用(Application)都可以在任何服务器(Server)上运行
+ 命令django-admin startproject会生成一个简单的wsgi.py文件，确定了settings、application对象
+ application对象：在Python模块中使用application对象与应用服务器交互
+ settings模块：Django需要导入settings模块，这里是应用定义的地方
+ 此处的服务器是一个软件，可以监听网卡端口、遵从网络层传输协议，收发http协议级别的数据

#### uWSGI
+ uWSGI实现了WSGI的所有接口，是一个快速、自我修复、开发人员和系统管理员友好的服务器
+ uWSGI代码完全用C编写
+ 安装uWSGI
+ pip install uwsgi<br><br>

+ 配置uWSGI，在项目中新建文件uwsgi.ini，编写如下配置
+ [uwsgi]
+ socket=外网ip:端口（使用nginx连接时，使用socket）
+ http=外网ip:端口（直接做web服务器，使用http）
+ chdir=项目根目录
+ wsgi-file=项目中wsgi.py文件的目录，相对于项目根目录
+ processes=4
+ threads=2
+ master=True
+ pidfile=uwsgi.pid
+ daemonize=uswgi.log<br><br>

+ 启动：uwsgi --ini uwsgi.ini
+ 停止：uwsgi --stop uwsgi.pid
+ 重启：uwsgi --reload uwsgi.pid
+ 使用http协议查看网站运行情况，运行正常，但是静态文件无法加载

#### nginx
+ 使用nginx的作用
+ 负载均衡：多台服务器轮流处理请求
+ 反射代理：隐藏真实服务器
+ 实现构架：客户端请求nginx，再由nginx请求uwsgi，运行django框架下的python代码
+ nginx+uwsgi也可以用于其它框架的python web代码，不限于django<br><br>

##### 负载均衡
+ 这里提到的客户端发送的、Nginx反向代理服务器接收到的请求数量，就是我们说的负载量。请求数量按照一定的规则进行分发到不同的服务器处理的规则，就是一种均衡规则。将服务器接收到的请求按照规则分发的过程，称为负载均衡。

###### Nginx支持的负载均衡调度算法方式如下：
+ weight轮询(默认，常用)：接收到的请求按照权重分配到不同的后端服务器，即使在使用过程中，某一台后端服务器宕机，Nginx会自动将该服务器剔除出队列，请求受理情况不会受到任何影响。 这种方式下，可以给不同的后端服务器设置一个权重值(weight)，用于调整不同的服务器上请求的分配率；权重数据越大，被分配到请求的几率越大；该权重值，主要是针对实际工作环境中不同的后端服务器硬件配置进行调整的。<br><br>

+ ip_hash（常用）：每个请求按照发起客户端的ip的hash结果进行匹配，这样的算法下一个固定ip地址的客户端总会访问到同一个后端服务器，这也在一定程度上解决了集群部署环境下session共享的问题。<br><br>

+ fair：智能调整调度算法，动态的根据后端服务器的请求处理到响应的时间进行均衡分配，响应时间短处理效率高的服务器分配到请求的概率高，响应时间长处理效率低的服务器分配到的请求少；结合了前两者的优点的一种调度算法。但是需要注意的是Nginx默认不支持fair算法，如果要使用这种调度算法，请安装upstream_fair模块。 <br><br>

+ url_hash：按照访问的url的hash结果分配请求，每个请求的url会指向后端固定的某个服务器，可以在Nginx作为静态服务器的情况下提高缓存效率。同样要注意Nginx默认不支持这种调度算法，要使用的话需要安装Nginx的hash软件包。

##### 反向代理
+ 多个客户端给服务器发送的请求，Nginx服务器接收到之后，按照一定的规则分发给了后端的业务处理服务器进行处理了。此时~请求的来源也就是客户端是明确的，但是请求具体由哪台服务器处理的并不明确了，Nginx扮演的就是一个反向代理角色。
+ 客户端是无感知代理的存在的，反向代理对外都是透明的，访问者并不知道自己访问的是一个代理。因为客户端不需要任何配置就可以访问。
+ 反向代理，"它代理的是服务端，代服务端接收请求"，主要用于服务器集群分布式部署的情况下，反向代理隐藏了服务器的信息。<br><br>

+ 反向代理的作用：
+ （1）保证内网的安全，通常将反向代理作为公网访问地址，Web服务器是内网
+ （2）负载均衡，通过反向代理服务器来优化网站的负载


+ 到官网下载nginx压缩文件或通过命令安装
+ sudo apt-get nginx<br><br>

+ 这里以下载压缩文件为例演示
+ 解压缩：
+ tar zxvf nginx-1.6.3.tar.gz<br><br>

+ 进入nginx-1.6.3目录依次执行如下命令进行安装：
+ ./configure
+ make
+ sudo make install<br><br>

+ 默认安装到/usr/local/nginx目录，进入此目录执行命令
+ 查看版本：sudo sbin/nginx -v
+ 启动：sudo sbin/nginx
+ 停止：sudo sbin/nginx -s stop
+ 重启：sudo sbin/nginx -s reload
+ 通过浏览器查看nginx运行结果
+ 指向uwsgi项目：编辑conf/nginx.conf文件<br><br>

+ sudo conf/nginx.conf
+ 在server下添加新的location项，指向uwsgi的ip与端口
+ location / {
+     include uwsgi_params;将所有的参数转到uwsgi下
+     uwsgi_pass uwsgi的ip与端口;
+ }
+ 修改uwsgi.ini文件，启动socket，禁用http
+ 重启nginx、uwsgi
+ 在浏览器中查看项目，发现静态文件加载不正常，接下来解决静态文件的问题

### 静态文件
+ 静态文件一直都找不到，现在终于可以解决了
+ 所有的静态文件都会由nginx处理，不会将请求转到uwsgi
+ 配置nginx的静态项，打开conf/nginx.conf文件，找到server，添加新location
+ location /static {
+     alias /var/www/test5/static/;
+ }
+ 在服务器上创建目录结构“/var/www/test5/”
+ 修改目录权限
+ sudo chmod 777 /var/www/test5
+ 创建static目录，注意顺序是先分配权限，再创建目录
+ mkdir static
+ 最终目录结构如下图：
+ 
+ 修改settings.py文件
+ STATIC_ROOT='/var/www/test5/static/'
+ STATIC_URL='/static/'
+ 收集所有静态文件到static_root指定目录：python manage.py collectstatic
+ 重启nginx、uwsgi